
# Dummy XGBoost Regressor


In [24]:
import numpy as np 
import pandas as pd 
import xgboost as xg 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 
  
# Load the data 
dataset = pd.read_csv("boston_housing.csv") 
X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1] 
  
# Splitting 
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                      test_size = 0.3, random_state = 123) 
  
# Instantiation 
# xgb_r = xg.XGBRegressor(objective ='reg:linear', 
#                   n_estimators = 10, seed = 123) 
xgb_r = xg.XGBRegressor(objective ='reg:squarederror', 
                  n_estimators = 10, seed = 123) 
  
# Fitting the model 
xgb_r.fit(train_X, train_y) 
  
# Predict the model 
pred = xgb_r.predict(test_X) 
  
# RMSE Computation 
rmse = np.sqrt(MSE(test_y, pred)) 
print("RMSE : % f" %(rmse)) 


RMSE :  3.713366


In [25]:
xgb_r.get_booster().save_config()

'{"learner":{"generic_param":{"fail_on_invalid_gpu_id":"0","gpu_id":"-1","n_jobs":"0","nthread":"0","random_state":"123","seed":"123","seed_per_iteration":"0","validate_parameters":"1"},"gradient_booster":{"gbtree_model_param":{"num_parallel_tree":"1","num_trees":"10","size_leaf_vector":"0"},"gbtree_train_param":{"predictor":"auto","process_type":"default","tree_method":"exact","updater":"grow_colmaker,prune","updater_seq":"grow_colmaker,prune"},"name":"gbtree","specified_updater":false,"updater":{"grow_colmaker":{"colmaker_train_param":{"default_direction":"learn","opt_dense_col":"1"},"train_param":{"alpha":"0","cache_opt":"1","colsample_bylevel":"1","colsample_bynode":"1","colsample_bytree":"1","eta":"0.300000012","gamma":"0","grow_policy":"depthwise","interaction_constraints":"","lambda":"1","learning_rate":"0.300000012","max_bin":"256","max_cat_threshold":"64","max_cat_to_onehot":"4","max_delta_step":"0","max_depth":"6","max_leaves":"0","min_child_weight":"1","min_split_loss":"0","

In [26]:
import pandas as pd
df = xgb_r.get_booster().trees_to_dataframe()

In [27]:
# GENERATE EACH TREE DATAFRAME

df0 = df[df["Tree"] == 0]
df1 = df[df["Tree"] == 1]
df2 = df[df["Tree"] == 2]
df3 = df[df["Tree"] == 3]
df4 = df[df["Tree"] == 4]
df5 = df[df["Tree"] == 5]
df6 = df[df["Tree"] == 6]
df7 = df[df["Tree"] == 7]
df8 = df[df["Tree"] == 8]
df9 = df[df["Tree"] == 9]

df0.columns

Index(['Tree', 'Node', 'ID', 'Feature', 'Split', 'Yes', 'No', 'Missing',
       'Gain', 'Cover', 'Category'],
      dtype='object')

In [28]:
def treeCalc(data, feature, threshold, childrenLeft, childrenRight, value):
    i = 0
    while feature[i] != -2:
        if data[feature[i]] < threshold[i]:
            i = childrenLeft[i]
        else:
            i = childrenRight[i]
    return value[i]

# Python PRED

In [29]:
# Input data
test_X[0:1]

,0,1,2,3,4,5,6,7,8,9,10,11,12
410,51.1358,0.0,18.1,0,0.597,5.757,100.0,1.413,24,666,20.2,2.6,10.11


In [30]:
pred = xgb_r.predict(test_X[0:1]) 
pred

array([29.097708], dtype=float32)

# C-Like PRED

In [31]:
# Same input data
testCase = [51.13580,0.0,18.10,0,0.5970,5.757,100.0,1.4130,24,666,20.2,2.60,10.11]

In [32]:
elem = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]
results = []

ft = []
th = []
cl = []
cr = []
vl = []

for el in elem:
    
    threshold = el.Split.replace(np.nan,-1.0).tolist()
    el['Yess'] = el['Yes'].str[2:]
    el['Noo'] = el['No'].str[2:]
    childrenRight = el.Noo.replace(np.nan,-1).astype(int).tolist()
    childrenLeft = el.Yess.replace(np.nan,-1).astype(int).tolist()
    value = el.Gain.tolist()
    feature = el.Feature.replace('Leaf',-2).astype(int).tolist()
    
    ft.append(feature)
    th.append(threshold)
    cl.append(childrenLeft)
    cr.append(childrenRight)
    vl.append(value)
    
    # calculation
    results.append(treeCalc(testCase, feature, threshold, childrenLeft, childrenRight, value))

bias = 0.5
print(np.sum(results) + bias)

29.097708245999996


C:\Users\Renan\AppData\Local\Temp\ipykernel_25036\3608822316.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  el['Yess'] = el['Yes'].str[2:]
C:\Users\Renan\AppData\Local\Temp\ipykernel_25036\3608822316.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  el['Noo'] = el['No'].str[2:]
C:\Users\Renan\AppData\Local\Temp\ipykernel_25036\3608822316.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [33]:
# # DEBUG
# print(childrenRight)
# print(childrenLeft)
# print(feature)
# print(value)
# print(threshold)

# C-Parser

In [34]:
name = "model"
for i in range(0,len(ft)):
    
    sft = "int " + name + "Feature" + str(i) + "[" + str(len(ft[i])) + "] = {"
    sth = "int " + name + "Threshold" + str(i) + "[" + str(len(th[i])) + "] = {"
    scl = "int " + name + "ChildrenLeft" + str(i) + "[" + str(len(cl[i])) + "] = {"
    scr = "int " + name + "ChildrenRight" + str(i) + "[" + str(len(cr[i])) + "] = {"
    svl = "int " + name + "Value" + str(i) + "[" + str(len(vl[i])) + "] = {"
    
    for j in range(0,len(ft[i])):
        sft = sft + str(ft[i][j]) + ", "
        sth = sth + str(th[i][j]) + ", "
        scl = scl + str(cl[i][j]) + ", "
        scr = scr + str(cr[i][j]) + ", "
        svl = svl + str(vl[i][j]) + ", "
        
    sft = sft[:len(sft)-2]
    sth = sth[:len(sth)-2]
    scl = scl[:len(scl)-2]
    scr = scr[:len(scr)-2]
    svl = svl[:len(svl)-2]
    
    sft += "};"
    sth += "};"
    scl += "};"
    scr += "};"
    svl += "};"
    
    print(sft)
    print(sth)
    print(scl)
    print(scr)
    print(svl)
    print()

int modelFeature0[19] = {5, 12, 5, 12, 12, 0, -2, -2, -2, 0, 0, -2, -2, -2, -2, 2, -2, -2, -2};
int modelThreshold0[19] = {6.97799969, 14.2600002, 7.43700027, 9.72500038, 21.4899998, 6.98962498, -1.0, -1.0, -1.0, 15.72505, 24.7199516, -1.0, -1.0, -1.0, -1.0, 26.6949997, -1.0, -1.0, -1.0};
int modelChildrenLeft0[19] = {1, 3, 5, 7, 9, 11, -1, -1, -1, 13, 15, -1, -1, -1, -1, 17, -1, -1, -1};
int modelChildrenRight0[19] = {2, 4, 6, 8, 10, 12, -1, -1, -1, 14, 16, -1, -1, -1, -1, 18, -1, -1, -1};
int modelValue0[19] = {13252.0938, 5099.58594, 874.132812, 442.046875, 627.511719, 32.0292969, 13.1112509, 7.43889904, 6.03202724, 83.2988281, 94.0341797, 9.30777836, 2.59500003, 4.95896149, 2.61000013, 7.27148438, 1.6500001, 3.50142884, 1.41000009};

int modelFeature1[25] = {5, 12, 5, 7, 0, 4, -2, -2, 5, 2, 0, -2, -2, -2, -2, 0, -2, 12, -2, -2, -2, -2, 4, -2, -2};
int modelThreshold1[25] = {6.79699993, 14.8600006, 7.43700027, 1.47134995, 5.76920986, 0.659000039, -1.0, -1.0, 6.06150007, 26.6949997, 